In [1]:
# define libraries to use
import pandas as pd
import numpy as np
from pathlib import Path
import requests,json
import matplotlib.pyplot as plt
%matplotlib inline
import time

In [2]:
#url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/';
#post_fields = {'format': 'json', 'data':'3GNDA13D76S000000;5XYKT3A12CG000000'};
#r = requests.post(url, data=post_fields);
#print(r.text);	

In [3]:
#print(r.json()["Results"][0])

In [4]:
"""
vpicFieldToKeep = [
	["ForwardCollisionWarningId","ForwardCollisionWarning"],
	["DynamicBrakeSupportId","DynamicBrakeSupport"],
	["CrashImminentBrakingID","CrashImminentBraking"],
	["CrashImminentBrakingID","CIB"],
	["PedestrianAutoEmergencyBrakingId","PedestrianAutoEmergencyBraking"],
	["PedestrianAutoEmergencyBrakingId","PedestrianAutomaticEmergencyBraking"],
	["BlindSpotWarningId","BlindSpotWarning"],
	["BlindSpotWarningId","BlindSpotMon"],
	["BlindSpotInterventionId","BlindSpotIntervention"],
	["LaneDepartureWarningId","LaneDepartureWarning"],
	["LaneKeepingAssistanceId","LaneKeepingAssistance"],
	["LaneKeepingAssistanceId","LaneKeepSystem"],
	["LaneCenteringAssistanceId","LaneCenteringAssistance"],
	["BackupCameraId","BackupCamera"],
	["BackupCameraId","RearVisibilitySystem"],
	["RearCrossTrafficAlertId","RearCrossTrafficAlert"],
	["RearAutomaticEmergencyBrakingId","RearAutomaticEmergencyBraking"],
	["ParkAssistId","ParkAssist"],
	["DaytimeRunningLightId","DaytimeRunningLight"],
	["HeadlampLightSourceId","HeadlampLightSource"],
	["HeadlampLightSourceId","LowerBeamHeadlampLightSource"],
	["SemiAutoHeadlampBeamSwitchingId","SemiAutoHeadlampBeamSwitching"],
	["SemiAutoHeadlampBeamSwitchingId","SemiautomaticHeadlampBeamSwitching"],
	["AdaptiveDrivingBeamId","AdaptiveDrivingBeam"],
	["AdaptiveCruiseControlId","AdaptiveCruiseControl"],
	["AntilockBrakeSystemId","AntilockBrakeSystem"],
	["AntilockBrakeSystemId","ABS"],
	["ElectronicStabilityControlId","ElectronicStabilityControl"],
	["ElectronicStabilityControlId","ESC"],
	["TPMSId","TPMS"],
	["TractionControlId","TractionControl"],
	["AutoPedestrianAlertingSoundId","AutoPedestrianAlertingSound"],
	["AutoPedestrianAlertingSoundId","AutomaticPedestrianAlertingSound"],
	["vpicVIN","VIN"]
]
"""

vpicFieldToKeep = [
	"ForwardCollisionWarning",
	"DynamicBrakeSupport",
	#"CrashImminentBraking",
	"CIB",
	#"PedestrianAutoEmergencyBraking",
	"PedestrianAutomaticEmergencyBraking",
	#"BlindSpotWarning",
	"BlindSpotMon",
	"BlindSpotIntervention",
	"LaneDepartureWarning",
	#"LaneKeepingAssistance",
	"LaneKeepSystem",
	"LaneCenteringAssistance",
	#"BackupCamera",
	"RearVisibilitySystem",
	"RearCrossTrafficAlert",
	"RearAutomaticEmergencyBraking",
	"ParkAssist",
	"DaytimeRunningLight",
	#"HeadlampLightSource",
	"LowerBeamHeadlampLightSource",
	#"SemiAutoHeadlampBeamSwitching",
	"SemiautomaticHeadlampBeamSwitching",
	"AdaptiveDrivingBeam",
	"AdaptiveCruiseControl",
	#"AntilockBrakeSystem",
	"ABS",
	#"ElectronicStabilityControl",
	"ESC",
	"TPMS",
	"TractionControl",
	#"AutoPedestrianAlertingSound",
	"AutomaticPedestrianAlertingSound",
	"VIN",
	"BodyClass",
	"Make",
	"MakeID",
	"Model",
	"ModelID",
	"ModelYear",
	"ErrorCode",
]



In [5]:
#print(r.json()["Results"][1].keys())
#for x in vpicFieldToKeep:
#    if x in r.json()["Results"][1].keys():
#        print(x,r.json()["Results"][0][x])
#    else:
#        print(x,"N/A")

In [6]:
vinDf = pd.read_csv("tmp/NYDMV-VIN.csv")
vinDf.columns = ["entry","ORIG_VIN"]
for x in vpicFieldToKeep:
    vinDf[x] = None
vinDf.set_index("entry", inplace=True)
vinDf.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11350911 entries, 2 to 12586700
Data columns (total 32 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   ORIG_VIN                             object
 1   ForwardCollisionWarning              object
 2   DynamicBrakeSupport                  object
 3   CIB                                  object
 4   PedestrianAutomaticEmergencyBraking  object
 5   BlindSpotMon                         object
 6   BlindSpotIntervention                object
 7   LaneDepartureWarning                 object
 8   LaneKeepSystem                       object
 9   LaneCenteringAssistance              object
 10  RearVisibilitySystem                 object
 11  RearCrossTrafficAlert                object
 12  RearAutomaticEmergencyBraking        object
 13  ParkAssist                           object
 14  DaytimeRunningLight                  object
 15  LowerBeamHeadlampLightSource         object
 16

In [13]:
#for idx in range(0,vinDf.shape[0],50):
url = 'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVINValuesBatch/';
vinStep = 48
vinRecords = vinDf.shape[0]
vinStart = 214560
vinStop = 1000000
for idx in range(vinStart,vinStop,vinStep):
    print("Current index:",idx,round(idx/vinRecords,1),"%",end="\r")
    if idx + vinStep < vinRecords:
        vinsToCheck = vinDf.iloc[idx:idx+vinStep,0]
    else:
        vinsToCheck = vinDf.iloc[idx:,0]
    vinsToCheckStr = vinsToCheck.to_csv(None,index=False,line_terminator=';',header=False)[:-1] # eliminate last ';'
    #print(vinsToCheckStr)
    post_fields = {'format': 'json', 'data':vinsToCheckStr};
    r = requests.post(url, data=post_fields);
    vpicResults = r.json()["Results"]
    # loop through resutls for every VIN

    for resIdx in range(0,len(vpicResults)):
        #rowLabel = vinDf.index[idx + resIdx]
        for x in vpicFieldToKeep:
            #vinDf.at[rowLabel,x] = vpicResults[resIdx][x]
            vinDf.iat[idx + resIdx,vinDf.columns.get_loc(x)] = vpicResults[resIdx][x]
            #print((rowLabel,x),(resIdx,x))
    
    #time.sleep(0.05)

    

In [12]:
print(vinDf.columns.get_loc("PedestrianAutomaticEmergencyBraking"))

4


In [ ]:
#print(r.json()["Results"])
vinDf.to_csv("tmp/NYDMV-VIN-OUTPUT.csv",index=True)

In [8]:
print(r.text)

ï»¿<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="utf-8">
	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<meta name="viewport" content="width=device-width, initial-scale=1">

	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	<meta name="title" content="vPIC (Vehicle product Information Catalog) by NHTSA" />

	<meta name="keywords" content="NHTSA, NHTSA vPIC, vPIC: Product Information Catalog and Vehicle Listing, VIN Decoder, Manufacturer Portal: Online Web-based Submittal Center, MID: Manufacturer Information Database, Modifiers: Adapted Vehicle Modifier Identification Database, API: Application Programming Interface" />
	<meta name="description" content="The NHTSA Product Information Catalog and Vehicle Listing (vPIC) is a consolidated platform that presents data collected within the manufacturer reported data from CFR 49 Parts 551 â 574 for use in a variety of modern tools. NHTSAâs vPIC platform is intended to serve as a centralized source 